# Data Processing Notebook

TIL Python Project group 9 
* Tessa van de Hulst (4963601)
* Maartje van den Broek (4964837)
* Lara de Geus (4965868)
* Pien Biersteker (4888375)

For this project we are going to look at the relationship between fuel prices for petrol, diesel and LPG and the average number of kilometers driven by a car. But before this is explored, the data files must be put into the correct format. Getting the data correct will be done in this notebook.

### Import necessary Libraries

In [13]:
# First, the necessary libraries were imported.
import pandas as pd
import numpy as np
import math
import scipy
import itertools

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

import plotly.io as pio

## Used Datasets:

There are two data sets used in this research. One data set contains the daily fuel prices from 2006 to 2019 and the other data set contains the traffic performance of passenger cars from 2006 to 2019. This time span is chosen because the fuel prices data set contains data from January 1st 2006 until the end of October 2022, but the traffic performance data set only contains data until 2019. The two data sets are retrieved from: 

   * Fuel prices data set 2006 til 2019
       - https://opendata.cbs.nl/statline/portal.html?_la=nl&_catalog=CBS&tableId=80416ned&_theme=426 

   * Traffic performance data set 2006 til 2019
       - https://opendata.cbs.nl/#/CBS/nl/dataset/80428ned/table

### Fuel prices data set

First, the fuel prices data is imported and adjusted.

In [14]:
# Import fuel data
file_path = 'downloads/fuelprices 2006-2019.csv'
df_fuel = pd.read_csv(file_path, delimiter = ';')

# In this piece of code the dutch column names are replaced for english column names
df_fuel.rename(columns ={'Perioden': 'Date', 
                         'BenzineEuro95_1': 'Petrol', 
                         'Diesel_2': 'Diesel', 
                         'Lpg_3': 'LPG'}, inplace=True)
df_fuel.head()

,Date,Petrol,Diesel,LPG
0,01-01-2006,1.325,1.003,0.543
1,02-01-2006,1.328,1.007,0.542
2,03-01-2006,1.332,1.007,0.540
3,04-01-2006,1.348,1.020,0.550
4,05-01-2006,1.347,1.021,0.550


In the table above, the average prices of the three different fuel types are given per day from January 1st 2006 until Decembre 31st 2019. In order to work with this data, the average prices per day are converted to the average prices per year (2006-2019). The calculation of the yearly average prices is shown in the code below.

In [15]:
# In this piece of code the average fuel price per day is converted to the average fuel price per year for all fuel types
average_petrol = df_fuel.groupby(pd.PeriodIndex(df_fuel['Date'], freq="Y"))['Petrol'].mean()
average_diesel = df_fuel.groupby(pd.PeriodIndex(df_fuel['Date'], freq="Y"))['Diesel'].mean()
average_LPG = df_fuel.groupby(pd.PeriodIndex(df_fuel['Date'], freq="Y"))['LPG'].mean()
averageFuel = df_fuel.groupby(pd.PeriodIndex(df_fuel['Date'], freq="Y"))['Petrol','Diesel','LPG'].mean()

averageFuel.head()


,Petrol,Diesel,LPG
Date,,,
2006,1.373255,1.043211,0.520479
2007,1.414156,1.059721,0.538271
2008,1.476393,1.241314,0.598790
2009,1.354011,1.012929,0.509482
2010,1.503186,1.170838,0.644268


To eventually include the yearly fuel prices into the other data set, the prices have to be listed under each other. This is the case because in the traffic performance data set, a distincion is made between the total number of cars, the number of cars on name of a company and the number of private cars. The fuel prices must therefore be listed under each other three times, so that the fuel prices are available for each property type. 

In [16]:
# In this piece of code a list is created to get the average prices for petrol, diesel and LPG under each other
average_f = list(itertools.chain(average_petrol, average_diesel, average_LPG))

# Next, the list is listed under each other three times, to fit into the traffic performance data set
averageFuel = list(itertools.chain(average_f, average_f, average_f))

### Traffic performance data set

Next, the traffic performance data set is imported and made suitable for the research.

In [5]:
# Import traffic performance data
file_p = 'downloads/traffic_data 2006-2019.csv'
df_traffic = pd.read_csv(file_p, delimiter = ';')

# Remove unnecessary columns 
df_traffic = df_traffic.drop(columns = ['Leeftijd voertuig', 'Kilometers personenauto\'s in Nederland/Totaal kilometers in Nederland (x mln km)',
                                        'Kilometers personenauto\'s in Nederland/Kilometers door Nederlandse voertuigen (x mln km)',
                                        'Kilometers personenauto\'s in Nederland/Kilometers door buitenlandse voertuigen (x mln km)',
                                        'Kilometers Nederlandse personenauto\'s/Totaal kilometers (x mln km)', 
                                        'Kilometers Nederlandse personenauto\'s/Kilometers in Nederland (x mln km)', 
                                        'Kilometers Nederlandse personenauto\'s/Kilometers in het buitenland (x mln km)', 
                                        'Gemiddeld jaarkilometrage/Totaal gemiddeld jaarkilometrage (aantal km)', 
                                        'Gemiddeld jaarkilometrage/Gemiddeld jaarkilometrage in buitenland (aantal km)',
                                        'Nederlandse personenauto\'s in gebruik (aantal)'] )

# Rename dutch column names to english column names
df_traffic.rename(columns ={'Eigendomssituatie': 'Property', 
                            'Brandstofsoort': 'Fuel type', 
                            'Perioden': 'Date', 
                            'Gemiddeld jaarkilometrage/Gemiddeld jaarkilometrage in Nederland (aantal km)': 'Annual mileage in the Netherlands (km)'}, inplace=True)

# Rename dutch cell values to english cell values.
df_traffic['Property'] = df_traffic['Property'].replace({'Totaal': 'Total', 'Bedrijf': 'Company', 'Particulier':'Private'})
df_traffic['Fuel type'] = df_traffic['Fuel type'].replace({'Benzine/overige': 'Petrol'})

# Remove cells with the value 'Totaal' in column Fuel type 
df_traffic = df_traffic[df_traffic['Fuel type'] != 'Totaal']

df_traffic

,Property,Fuel type,Date,Annual mileage in the Netherlands (km)
14,Total,Petrol,2006,9711
15,Total,Petrol,2007,9667
16,Total,Petrol,2008,9359
17,Total,Petrol,2009,9379
18,Total,Petrol,2010,9369
...,...,...,...,...
163,Company,LPG,2015,16271
164,Company,LPG,2016,15788
165,Company,LPG,2017,15863
166,Company,LPG,2018,15604


It is chosen to only look at the annual mileage in the Netherlands for answering the research questions. Therefore  a couple of columns from the traffic performance data set are irrelevant and hence excluded. Additionallly, the rows containing 'Total' for Fuel type are also excluded because the total fuel price does not provide relevant information. In the fuel prices data set it can be seen that the prices of the different fuel types differ a lot, making it irrelevant to include the total fuel price, because this is an average of the three different fuel prices. This means the fuel type 'Total' is irrelevant and therefore excluded from the data set.

The annual mileage in the Netherlands implies the average number of kilometers driven by a car in the Netherlands. Hereby, it is chosen to only include the cars driving in the Netherlands with a dutch license plate. So the kilometers driven abroad are excluded, and therefore the total number of kilometers driven by a dutch car is excluded as well, because this consist of the kilometers driven in the Netherlands and abroad. 

In order to distinguish the number of kilometers driven by company cars and private cars, the column 'Property' is seperated into two new columns, 'Annual Mileage Company (km)' and 'Annual Mileage Private (km)'. The column 'Annual Mileage Company (km)' only contains the number of driven kilometers with a company car. The rest of the cells are empty, labelled as NaN. The same is done for the other added column 'Annual Mileage Private (km)'. The results can be seen in the table below.  

In [6]:
# In this piece of code a column is added for the annual mileage of company and private cars in km
df_traffic['Annual Mileage Company (km)'] = np.where(df_traffic['Property'] == 'Company', df_traffic['Annual mileage in the Netherlands (km)'], np.nan) 
df_traffic['Annual Mileage Private (km)'] = np.where(df_traffic['Property'] == 'Private', df_traffic['Annual mileage in the Netherlands (km)'], np.nan)

# Now the dataset has seperate columns for different property 
df_traffic

,Property,Fuel type,Date,Annual mileage in the Netherlands (km),Annual Mileage Company (km),Annual Mileage Private (km),Annual mileage Petrol (km),Annual mileage Diesel (km),Annual mileage LPG (km)
14,Total,Petrol,2006,9711,NaN,NaN,9711.0,NaN,NaN
15,Total,Petrol,2007,9667,NaN,NaN,9667.0,NaN,NaN
16,Total,Petrol,2008,9359,NaN,NaN,9359.0,NaN,NaN
17,Total,Petrol,2009,9379,NaN,NaN,9379.0,NaN,NaN
18,Total,Petrol,2010,9369,NaN,NaN,9369.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...
163,Company,LPG,2015,16271,16271.0,NaN,NaN,NaN,16271.0
164,Company,LPG,2016,15788,15788.0,NaN,NaN,NaN,15788.0
165,Company,LPG,2017,15863,15863.0,NaN,NaN,NaN,15863.0
166,Company,LPG,2018,15604,15604.0,NaN,NaN,NaN,15604.0


## Combine Data sets Fuelprices & Traffic use

In order to answer the research questions, the two data sets must be combined. To combine the data sets, the averageFuel column, which consists of the listed yearly fuel prices, is added to the traffic performance data set.

In [7]:
# The average fuel prices list which is made before is added to the df_Traffic dataframe.
df_traffic['Fuel price'] = averageFuel 

df_CombinedDatasets = df_traffic

df_CombinedDatasets

,Property,Fuel type,Date,Annual mileage in the Netherlands (km),Annual Mileage Company (km),Annual Mileage Private (km),Annual mileage Petrol (km),Annual mileage Diesel (km),Annual mileage LPG (km),Fuel price
14,Total,Petrol,2006,9711,NaN,NaN,9711.0,NaN,NaN,1.373255
15,Total,Petrol,2007,9667,NaN,NaN,9667.0,NaN,NaN,1.414156
16,Total,Petrol,2008,9359,NaN,NaN,9359.0,NaN,NaN,1.476393
17,Total,Petrol,2009,9379,NaN,NaN,9379.0,NaN,NaN,1.354011
18,Total,Petrol,2010,9369,NaN,NaN,9369.0,NaN,NaN,1.503186
...,...,...,...,...,...,...,...,...,...,...
163,Company,LPG,2015,16271,16271.0,NaN,NaN,NaN,16271.0,0.618951
164,Company,LPG,2016,15788,15788.0,NaN,NaN,NaN,15788.0,0.571298
165,Company,LPG,2017,15863,15863.0,NaN,NaN,NaN,15863.0,0.632584
166,Company,LPG,2018,15604,15604.0,NaN,NaN,NaN,15604.0,0.685408


## Save combined dataframe to a csv file 

In order to use the combined data set in the main notebook, where the research questions will be answered, the dataframe df_CombinedDatasets is saved into a new csv file.

In [8]:
# First the columns are defined for the new csv file
column_names = ['Property', 'Fuel type', 'Date', 'Annual mileage in the Netherlands (km)', 'Annual Mileage Company (km)','Annual Mileage Private (km)','Fuel price',
                'Annual mileage Petrol (km)', 'Annual mileage Diesel (km)','Annual mileage LPG (km)']
# Then the combined dataframe is saved as a csv file
df_CombinedDatasets.to_csv("downloads/total_data_traffic.csv",
                    na_rep='', columns=column_names, header=True, index=False)

Also, the adjusted fuel prices data set is saved into a new csv file because this data is needed for subquestion 1.

In [10]:
# Then the combined dataframe is saved into an excel file
df_fuel.to_csv("downloads/fueldata.csv",
                    na_rep='', header=True, index=False)